In [1]:

import pandas as pd
from torch.utils.data import Dataset

from torch.utils.data import DataLoader
import torch
from torchvision import models
import torch
import pandas as pd
from PIL import Image
import torch.nn as nn
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import pandas as pd

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
"""
テキスト処理のモデル
"""
class CaptionEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
  def forward(self, x):
    x = self.bert(x)
    x = torch.max(x.last_hidden_state, dim=1)[0]  # max pooling
    return x

In [4]:
model_name =  './model/model_caption_2023-06-28.pth'
caption_model = CaptionEncoder().to(device)
caption_model.load_state_dict(torch.load(model_name))
caption_model.eval()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CaptionEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [5]:
class FDataset(Dataset):
    def __init__(self, annotations_file):
        self.data = pd.read_csv(annotations_file) 
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        caption = self.data.iloc[idx, 1]
        return caption

In [19]:
dataset = FDataset('./data/anotation_new.csv')
dataloader =  DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True, num_workers=0)
caption_embeddings = torch.Tensor([])

In [8]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")

In [21]:
for i, (cap) in enumerate(dataloader):
    # 予測と損失の計算
    ids = tokenizer.encode(cap, return_tensors='pt')
    cap = ids.to(device)
    
    with torch.no_grad():
        caption_embedding = caption_model(cap).cpu()
    caption_embeddings = torch.cat((caption_embeddings, caption_embedding), dim=0)
    if i % 1000 == 0:
        print(f"finish: {len(caption_embeddings) * 100/len(dataset)}%")

torch.save(caption_embeddings, f'caption_tensor/model_caption_2023-06-28/bert_tensor.pt')

finish: 0.025110458466593252%
finish: 0.10382662920826488%
finish: 0.18254279994993652%
finish: 0.26125897069160814%
finish: 0.3399751414332798%


KeyboardInterrupt: 

In [ ]:
# もしかしたら、学習しなおしかも。encoderが、配列にも対応していて、

In [15]:
len(caption_embeddings)

10259